In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tifffile as tif
from aicsimageio import AICSImage
from aicsimageio.readers.ome_tiff_reader import OmeTiffReader
from aicsimageio.writers import OmeTiffWriter
import matplotlib.image as mpimg

import sys
src_path = str(Path.cwd().parent)
if src_path not in sys.path:
    sys.path.append(src_path)

from src.d01_init_proc import vis_and_rescale

In [ ]:
orig_fig_dirpath = Path(input())

In [ ]:
seg_dirpath = Path(input())

In [ ]:
# find incorrectly saved segmentations
todelete = []
for segpath in seg_dirpath.glob('*.tif'):
    seg = tif.imread(segpath)
    
    if np.max(seg) > 3:
        print(f'{segpath.name} not correctly segmented')
        todelete.append(segpath)
if len(todelete)==0:
    print('All imgs correctly segmented')

In [ ]:
# CAREFUL!!! Deletes incorrectly segmented images identified in previous cell
num_deleted = 0
for imgpath in todelete:
    imgpath.unlink()
    num_deleted = num_deleted + 1
print(f'{num_deleted} files deleted')

In [ ]:
# Move already segmented images to a new directory

orig_dirpath = orig_fig_dirpath.parent
orig_dirpath_done = orig_dirpath / 'segmented'
orig_dirpath_done.mkdir(parents=True, exist_ok=True)

orig_imgnames = [path.name.split('.')[0] for path in orig_dirpath.glob('*.ome.tif')]
seg_imgnames = [path.name.split('.')[0] for path in seg_dirpath.glob('*.tif')]

orig_imgnames_done = [name for name in orig_imgnames if (name+'_seg') in seg_imgnames]

num_moved = 0
for name in orig_imgnames_done:
    orig_imgpath = orig_dirpath / (name + '.ome.tif')
    if orig_imgpath.is_file():
        orig_imgpath.rename(orig_dirpath_done / (name + '.ome.tif'))
        num_moved = num_moved + 1
    else:
        print(f'{orig_imgpath.name} not found')        
            
print(f'{num_moved} files moved')
    

In [ ]:
#TODO: Separate out segmention figure generation
# Double check that this still works for images with 1 timepoint
# Add separate display function for both figures

seg_fig_dirpath = seg_dirpath / 'figs'
seg_fig_dirpath.mkdir(parents=True, exist_ok=True)

imgpaths = [imgpath for imgpath in seg_dirpath.glob('*.tif')]
imgpaths.sort()


for imgpath in imgpaths:
    img_file = AICSImage(imgpath)
    img = img_file.data
    size_t = img.shape[0]
    
    fig, axs = plt.subplots(size_t, 4, figsize=(16, 3*size_t), squeeze=False, constrained_layout=True)
    
    for t in range(size_t):
        axs[t, 0].imshow(img[t, 0, 0, :, :], interpolation=None, cmap='gray')
        axs[t, 0].axis('off')
        axs[t, 0].set_title(f'merged, t{t}')

        cell = (img > 0)
        axs[t, 1].imshow(cell[t, 0, 0, :, :], interpolation=None, cmap='gray')
        axs[t, 1].axis('off')
        axs[t, 1].set_title(f'cell, t{t}')

        caax_pos = (img==1)
        axs[t, 2].imshow(caax_pos[t, 0, 0, :, :], interpolation=None, cmap='gray')
        axs[t, 2].axis('off')
        axs[t, 2].set_title(f'caax+, t{t}')

        caax_neg = (img==2)
        axs[t, 3].imshow(caax_neg[t, 0, 0, :, :], interpolation=None, cmap='gray')
        axs[t, 3].axis('off')
        axs[t, 3].set_title(f'caax-, t{t}')
    
    imgname = imgpath.name.split('.')[0]
    fig.suptitle(imgname)
    plt.show()
    fig.savefig(seg_fig_dirpath / (imgname + '.png'))

    orig_fig_path = orig_fig_dirpath / (imgpath.name.split('_seg')[0] + '.png')
    if orig_fig_path.is_file():
        orig_fig = mpimg.imread(orig_fig_path)
        vis_and_rescale.display_fig(orig_fig)
    else:
        print(f'{orig_fig_path.name} not found')
print('Done!')
        